In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23832094
paper_name = 'tun_wu_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/c3mt00083d.xlsx', sheet_name='Sheet1', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5048 x 5


In [7]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [9]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [10]:
original_data.loc[original_data['ORF']=='JR078W','ORF'] = 'YJR078W'
original_data.loc[original_data['ORF']=='YOR205CHOMDIP','ORF'] = 'YOR205C'

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

                ORF    Control  Diamide 0.8mM  Al3+ 0.4 mM  \
index_input                                                  
0            BY4743  10.708712      10.780723     8.119284   

             Diamide 0.8 mM + Al3+ 0.4 mM  
index_input                                
0                                6.555099  


In [12]:
original_data.set_index('ORF', inplace=True)
original_data.index.name = 'orf'

In [13]:
# Normalize to Control
original_data.iloc[:,1:] = original_data.iloc[:,1:].div(original_data.iloc[:,0], axis=0)

In [14]:
# Normalize to WT
original_data = original_data.div(original_data.iloc[0,:], axis=1)

In [15]:
# Remove the WT
original_data = original_data.drop(index='BY4743')

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.shape

(4827, 4)

# Prepare the final dataset

In [19]:
data = original_data.copy()

In [20]:
dataset_ids = [16483, 16481, 16479, 16482]
datasets = datasets.reindex(index=dataset_ids)

In [21]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [22]:
data.head()

dataset_id,16483,16481,16479,16482
data_type,value,value,value,value
orf,,,,
YAL002W,0.829311,1.050944,0.942367,1.007451
YAL004W,0.774919,0.969998,1.245198,1.324430
YAL005C,0.822704,0.884366,1.113115,1.180247
YAL007C,0.832042,0.898153,1.087283,1.235286
YAL008W,0.747878,1.191389,1.158506,1.312689


## Subset to the genes currently in SGD

In [23]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [24]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16483,16481,16479,16482
,data_type,value,value,value,value
gene_id,orf,,,,
2,YAL002W,0.829311,1.050944,0.942367,1.007451
1863,YAL004W,0.774919,0.969998,1.245198,1.324430
4,YAL005C,0.822704,0.884366,1.113115,1.180247
5,YAL007C,0.832042,0.898153,1.087283,1.235286
6,YAL008W,0.747878,1.191389,1.158506,1.312689


# Normalize

In [25]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [26]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [27]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16483,16481,16479,16482,16483,16481,16479,16482
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
2,YAL002W,0.829311,1.050944,0.942367,1.007451,-1.185477,0.469692,-0.559835,-0.183301
1863,YAL004W,0.774919,0.969998,1.245198,1.324430,-1.645145,-0.136946,1.071390,1.109988
4,YAL005C,0.822704,0.884366,1.113115,1.180247,-1.241315,-0.778704,0.359916,0.521715
5,YAL007C,0.832042,0.898153,1.087283,1.235286,-1.162397,-0.675378,0.220768,0.746275
6,YAL008W,0.747878,1.191389,1.158506,1.312689,-1.873671,1.522245,0.604419,1.062083


# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/4 [00:00<?, ?it/s]

Deleting all datasets for PMID 23832094...
Inserting the new data...


100%|██████████| 4/4 [00:28<00:00,  7.13s/it]

Updating the data_modified_on field...
